In [17]:
import pandas as pd
import geopandas

In [300]:
explore_args = {
    "cmap": "Blues",
    "style_kwds": {
        "color": "black",
        "weight": 0.5,
        "opacity": 1,
        "fillOpacity": 0.8
    }
}

In [301]:
from copy import deepcopy

# I don't know why but when this is passed directly GeoDataFrame.explore removes the color
# element from style_kwds
def get_explore_args():
    return deepcopy(explore_args)

## Loading data

In [259]:
lor_gdf = geopandas.read_file("data/lor_friedrichshain_kreuzberg.geojson")
streets_processed_gdf = geopandas.read_file("data/extras/streets_processed.geojson")
drive_ways_gdf = geopandas.read_file("data/extras/drive_ways.geojson")
service_processed_gdf = geopandas.read_file("data/extras/service_processed.geojson")
estimated_parking_lanes_gdf = geopandas.read_file("data/parking/estimated_parking_lanes.geojson")
estimated_parking_spots_kfz_gdf = geopandas.read_file("data/parking/estimated_parking_spots_kfz.geojson")
demographics_age_df = pd.read_csv("data/demographics/demographics_age_of_inhabitants.csv", thousands=" ")
demographics_background_df = pd.read_csv("data/demographics/demographics_background_of_inhabitants.csv", thousands=" ", decimal=",")
registered_cars_gdf = geopandas.read_file("data/demographics/registered_cars.geojson")

## Merging some dataframes

In [260]:
demographics_df = pd.merge(demographics_age_df, demographics_background_df, on=("lor_number", "lor"))
demographics_df["lor"] = demographics_df["lor"].str.strip()

In [261]:
set(demographics_df["lor"].unique()).symmetric_difference(set(registered_cars_gdf["lor"].unique()))

{'Am Berlin Museum',
 'Forckenbeckplatz',
 'Gleisdreieck',
 'Gleisdreieck/Entwicklungsgebiet',
 'Graefekiez',
 'Graefekiez Nord',
 'Graefekiez Süd',
 'Köpenicker Straße',
 'Niederbarnim-straße',
 'Pettenkofer Straße',
 'Prinzenstraße',
 'Reichenberger Straße',
 'Reichenberger Straße Ost',
 'Reichenberger Straße West',
 'Revaler Straße',
 'Wismarplatz',
 'Wriezener Bahnhof',
 'Wriezener Bahnhof/Entwicklungsgebiet'}

## Visualizing data

In [262]:
def get_lor_map():
    return lor_gdf.explore(
        column="PLR_NAME",
        legend=True,
        legend_kwds={"colorbar": False},
        style_kwds={"opacity": 0.4, "fillOpacity": 0.4},
        name="Friedrichshain-Kreuzberg"
    )

In [264]:
estimated_parking_spots_kfz_lor_gdf = estimated_parking_spots_kfz_gdf.sjoin(lor_gdf, how="inner")

In [265]:
parking_spot_count_per_lor = estimated_parking_spots_kfz_lor_gdf \
    .groupby("PLR_NAME") \
    .size() \
    .to_frame("parking_spot_count") \
    .reset_index()

In [266]:
parking_spot_count_per_lor.head(5)

,PLR_NAME,parking_spot_count
0,Am Berlin Museum,983
1,Andreasviertel,3542
2,Askanischer Platz,2297
3,Barnimkiez,1397
4,Boxhagener Platz,1663


In [267]:
estimated_parking_spots_kfz_old_lor_gdf = estimated_parking_spots_kfz_gdf \
    .sjoin(registered_cars_gdf, how="inner")

In [268]:
parking_spot_count_per_old_lor = estimated_parking_spots_kfz_old_lor_gdf \
    .groupby("lor") \
    .size() \
    .to_frame("parking_spot_count") \
    .reset_index()

In [269]:
parking_spot_count_per_old_lor.head(5)

,lor,parking_spot_count
0,Andreasviertel,3799
1,Askanischer Platz,2301
2,Barnimkiez,1397
3,Boxhagener Platz,3697
4,Chamissokiez,2928


In [315]:
streets_processed_gdf

,highway,name,surface,estimated_width,geometry
0,primary,Frankfurter Allee,asphalt,17,"LINESTRING (395107.995 5819522.240, 395119.955..."
1,primary,Elsenbrücke,asphalt,10,"LINESTRING (395719.722 5817390.173, 395607.288..."
2,primary,Mehringdamm,asphalt,17,"LINESTRING (390659.766 5817422.138, 390660.195..."
3,primary,Warschauer Straße,asphalt,17,"LINESTRING (394817.686 5818611.941, 394830.610..."
4,residential,Kinzigstraße,asphalt,11,"LINESTRING (395876.398 5819185.526, 395871.266..."
...,...,...,...,...,...
2086,residential,Segitzdamm,sett,11,"LINESTRING (392299.323 5817786.329, 392275.413..."
2087,construction,None,None,11,"LINESTRING (395619.535 5817231.873, 395722.033..."
2088,construction,None,None,11,"LINESTRING (395612.371 5817237.355, 395715.095..."
2089,primary,Frankfurter Allee,asphalt,17,"LINESTRING (395914.043 5819387.153, 395909.336..."


## Visualize registered cars with old LOR (from 2019)

In [270]:
registered_cars_parking_spot_count_gdf = registered_cars_gdf \
    .merge(parking_spot_count_per_old_lor, on="lor")

In [271]:
registered_cars_parking_spot_count_gdf["vehicles_per_parking"] = \
    (registered_cars_parking_spot_count_gdf["vehicles_overall"]
    / registered_cars_parking_spot_count_gdf["parking_spot_count"]).apply(lambda x: round(x, 2))

registered_cars_parking_spot_count_gdf["cars_per_parking"] = \
    (registered_cars_parking_spot_count_gdf["cars_only"]
    / registered_cars_parking_spot_count_gdf["parking_spot_count"]).apply(lambda x: round(x, 2))

In [272]:
registered_cars_parking_spot_count_gdf["parking_spots_per_km2"] = \
    (registered_cars_parking_spot_count_gdf["parking_spot_count"]
    / (registered_cars_parking_spot_count_gdf["lor size in m²"] / 10 ** 6)).apply(lambda x: round(x, 2))

In [302]:
registered_cars_parking_spot_count_gdf.explore(
    column="cars_per_parking",
    **get_explore_args()
)

In [303]:
registered_cars_parking_spot_count_gdf.explore(
    column="vehicles_per_parking",
    **get_explore_args()
)

In [306]:
registered_cars_parking_spot_count_gdf.explore(
    column="parking_spots_per_km2",
    **get_explore_args()
)

## Visualize registered cars with new LOR taking only districts that stayed the same

In [307]:
stats_df = registered_cars_gdf.merge(demographics_df, on="lor")
stats_df.columns

Index(['district ', 'lor', 'Bezirksregion', 'Prognoseraum', 'lor size in m²',
       'inhabitants_total', 'of_those_inhabitants_18+', 'vehicles_overall',
       'cars_only', 'vehicles_per_1000_inhabitants',
       'cars_per_1000_inhabitants', 'geometry', 'lor_number', 'total',
       'below age 6', ' 6 - 15', '15 - 18', '18 - 27', '27 - 45', '45 - 55',
       '55 - 65', '65 an more', 'of those female', 'of those immigrants',
       'total_of_inhabitants', 'total_in_percent(%)',
       'of_those_with_migration_background',
       'of_those_with_migration_background in percent (%)',
       'of_German_heritage', 'of_German_heritage_in_percent(%)',
       'germans_without_migration_background',
       'germans_without_migration_background_in_percent (%)',
       'germans_with_a_migration_background',
       'germans_with_a_migration_background_in_percent (%)',
       'number_of_non-Germans', 'number_of_non-Germans_in_percent(%)'],
      dtype='object')

In [308]:
# Only use districts that didn't change much in size
stats_df = stats_df[abs(stats_df["inhabitants_total"] - stats_df["total"]) / stats_df["inhabitants_total"] < 0.1]

In [309]:
stats_df = stats_df \
    .merge(parking_spot_count_per_lor, left_on="lor", right_on="PLR_NAME")

In [310]:
stats_df["vehicles_per_parking"] = (stats_df["vehicles_overall"] / stats_df["parking_spot_count"]).apply(lambda x: round(x, 2))
stats_df["cars_per_parking"] = (stats_df["cars_only"] / stats_df["parking_spot_count"]).apply(lambda x: round(x, 2))

In [311]:
stats_df["parking_spots_per_km2"] = (stats_df["parking_spot_count"] / (stats_df["lor size in m²"] / 10 ** 6)).apply(lambda x: round(x, 2))

In [312]:
stats_df.explore(
    column="vehicles_per_parking",
    **get_explore_args()
)

In [313]:
stats_df.explore(
    column="cars_per_parking",
    **get_explore_args()
)

In [314]:
stats_df.explore(
    column="parking_spots_per_km2",
    **get_explore_args()
)

# TODO: Number of car registrations in comparison to street surface
# TODO: